# 참고. flask 와 데이터베이스
- 실제 웹서비스에서는 다양한 데이터베이스를 사용함
- 변경 여지가 적은 경우, mysql 과 같은 관계형 데이터베이스 사용
  - 가장 정보가 많고, 수십년간 사용되었기 때문에 보안, 안정적임
- 변경 여지가 큰 웹서비스 로깅(logging)을 위해서는 NoSQL 을 사용
- 이외에 웹서비스 성능 향상을 위해 redis 와 같은 in-memory 방식의 데이터베이스도 많이 사용됨
- mongodb 는 크롤링 데이터등과 같이 방대하지만, 일부 유실되도 큰 문제없는 데이터 처리시 더 적합
- 규격화되고 각 데이터의 극도의 안정성이 필요한 데이터는 MySQL 이 더 적합


---------------

# 주요DB기능 구현내용

### 블로그 기능과 데이터베이스 활용
- mysql: 구독으로 얻어진 이메일 주소 정보를 저장
- mongodb: 블로그 접근 사용자의 IP 와 접근 시간을 로그 형태로 저장 
  - logging 라이브러리로 파일로 저장하지 않고, 몽고db를 활용
  - 몽고db가 익숙치 않을 경우, logging 라이브러리로도 활용 가능
  - 로깅은 통상적으로 수시로 포맷이 바뀌는 경우가 많으므로, 스키마 설정이 필요없는 NoSQL 이 적합

-----------

# 어떤방식으로 파이썬환경에서 구현할것인가


### SQLAlchemy VS Pymysql
- Django 등에서 다양한 데이터베이스를 객체로 만들어서, 사용법을 통일하는 기법으로 SQLAlchemy 라는 기능을 사용함
- SQL 이외에 추가적인 SQLAlchemy 기반 CRUD 기법을 익혀야 함
- SQL, mongodb CRUD 에 익숙해지는 것이 보다 중요함


> pymysql 을 쓰기로 함

- 굳이 객체로 만들어서, 복잡하게 다룰 필요 없음
- 객체 생성이 느리므로, 성능도 직접 데이터베이스를 엑세스하는 것에 비해 나을 것이 없음

```
At the ORM level, the speed issues are because creating objects in Python is slow, and the SQLAlchemy ORM applies a large amount of bookkeeping to these objects as it fetches them, which is necessary in order for it to fulfill its usage contract, including unit of work, identity map, eager loading, collections, etc.
```

### pymysql 라이브러리 소개 및 설치
* mysql을 python에서 사용할 수 있는 라이브러리 (pymysql 라이브러리 이외에도 MySQLdb(Mysql-pytion), MySQL connector 등 다양한 라이브러리 존재)
* 이 중에서 설치가 가장 쉬운 라이브러리
* 설치
  - pip install PyMySQL
  
* 일반적인 mysql 핸들링 코드 작성 순서
  1. PyMySql 모듈 import
  2. pymysql.connect() 메소드를 사용하여 MySQL에 연결
     - 호스트명, 포트, 로그인, 암호, 접속할 DB 등을 파라미터로 지정
  3. MySQL 접속이 성공하면, Connection 객체로부터 cursor() 메서드를 호출하여 Cursor 객체를 가져옴
  4. Cursor 객체의 execute() 메서드를 사용하여 SQL 문장을 DB 서버에 전송
  5. SQL 쿼리의 경우 Cursor 객체의 fetchall(), fetchone(), fetchmany() 등의 메서드를 사용하여 서버로부터 가져온 데이타를 코드에서 활용
  6. 삽입, 갱신, 삭제 등의 DML(Data Manipulation Language) 문장을 실행하는 경우, INSERT/UPDATE/DELETE 후 Connection 객체의 commit() 메서드를 사용하여 데이타를 확정
  7. Connection 객체의 close() 메서드를 사용하여 DB 연결을 닫음

### Pymysql 로 mysql 접속
* pymysql.connect() 메소드를 사용하여 MySQL에 연결
     - 호스트명, 포트, 로그인, 암호, 접속할 DB 등을 파라미터로 지정
     - 주요 파라미터
       - host : 접속할 mysql server 주소
       - port : 접속할 mysql server 의 포트 번호
       - user : mysql ID
       - passwd : mysql ID의 암호
       - db : 접속할 데이터베이스
       - charset='utf8' : mysql에서 select하여 데이타를 가져올 때 한글이 깨질 수 있으므로 연결 설정에 넣어줌

--------------

# 사전 작업(디비설정 및 mysql서버실행)
```
mysql -u root
CREATE DATABASE blog_db;
```  
```
mysqladmin -u root -p status
```

### pymysql연결설정

In [4]:
import pymysql

db_conn = pymysql.connect(
        host='localhost', 
        port=3306, 
        user='root', 
        passwd='ryan1024@', 
        db='blog_db', 
        charset='utf8')

dave_db = db_conn.cursor()
dave_db

### 사용자 테이블 정의 mysql workbench사용
- USER_ID 로 사용자 정보를 접근
- USER_EMAIL 도 unique 해야 하지만 동일한 USER_EMAIL 은 입력 코드에서 체크하기로 함

```sql
CREATE TABLE user_info (
    USER_ID INT UNSIGNED NOT NULL AUTO_INCREMENT,
    USER_EMAIL VARCHAR(100) NOT NULL,
    BLOG_ID CHAR(4),
    PRIMARY KEY(USER_ID)
);
```

------------

### mysql 테스트(CRUD)

> 실제 flask 코드에 넣어서 테스트할 경우, 테스트에 굉장한 시간 소요
> 필요한 테스트는 주피터 노트북상에서 가능한 많이 한 후에, 최종 결과가 될 수 있는 코드만 flask 코드로 변환

### 기존 테이블 수 확인하기

In [5]:
sql = 'SHOW TABLES;'
dave_db.execute(sql)

1

### 테이블 삭제하기(DELETE)

In [8]:
sql = 'DROP TABLE user_info;'
dave_db.execute(sql)
db_conn.commit() # 데이터베이스를 변경하는 명령은 commit() 해주기

### 테이블 수 확인하기
- 테이블 생성되었으므로 테이블 수가 1이어야 맞음

In [9]:
sql = 'SHOW TABLES;'
dave_db.execute(sql)

0

### 테이블에 데이터 넣기

In [6]:
user_email = 'test@test.com' #이메일 형식 제한 제어 코드는 일단 생략
blog_id = 'A'

In [7]:
sql = "INSERT INTO user_info (USER_EMAIL, BLOG_ID) VALUES ('%s', '%s')" % (str(user_email), str(blog_id))
dave_db.execute(sql)
db_conn.commit() # 데이터베이스를 변경하는 명령은 commit() 해주기

### 테이블 데이터 조회(SELECT)
  - Cursor Object 가져오기: cursor = db.cursor()  
  - SQL 실행하기: cursor.execute(SQL)
  - mysql 서버로부터 데이터 가져오기: fetch 메서드 사용
    - fetchall(): Fetch all the rows
    - fetchmany(size=None): Fetch several rows
    - fetchone(): Fetch the next row

In [8]:
sql = "SELECT * FROM user_info"
dave_db.execute(sql)
results = dave_db.fetchall() #검색된 데이터 가져오기
for result in results:
    print (result, type(result))

(1, 'test@test.com', 'A') <class 'tuple'>


### USER_EMAIL 로 테이블 데이터 조회

In [9]:
user_id = 1

In [5]:
sql = "DELETE FROM user_info WHERE USER_ID = %d" % (user_id)
print(dave_db.execute(sql))
db_conn.commit() # 데이터베이스를 변경하는 명령은 commit() 해주기

0


### 데이터베이스 연결 해제

In [15]:
db_conn.close()

---------------
# 세션 로그 데이터 저장을 위한 Mongodb 설정

### Mongodb 연결

In [10]:
import pymongo

username = ''
password = ''
ip_address = 'localhost:27017'
connection = pymongo.MongoClient()
connection = pymongo.MongoClient('mongodb://%s' % (ip_address))
# connection = pymongo.MongoClient('mongodb://%s:%s@%s' % (username, password, ip_address))
blog_session_db = connection.blog_session_db
blog_ab = blog_session_db.blog_ab

### Mongodb 연결 확인
- mongodb 연결을 한번 해놓으면, 이론적으로는 해당 객체를 사용, 몽고db CRUD 를 실행하면 됨
- 하지만, 실제로는 mongodb 가 다양한 원인으로 다운되거나, 연결이 해제되는 경우가 있음
  - 이 경우, 연결이 되어있음을 가정하고, CRUD를 실행할 경우 에러가 남
- 따라서 연결된 객체가 아직 mongodb 에 연결이 되어있는지 체크하는 방법이 필요함
  - 물론, 이 방법도 mongodb 가 아예 다운된 경우등을 체크하도록 코드를 더 정교하게 작성할 수도 있지만,
  - 문제를 mongodb 는 다운되더라도 재기동된다고 가정하고, (docker 등 다른 시스템을 통해 해당 기능 활성화)
  - 연결이 해제된 경우만 체크하는 기법을 사용 
    - 특별한 방법이 없으므로, 가장 간단한 명령을 내려보는 방법으로 진행

In [11]:
connection.admin.command('ismaster')

{'ismaster': True,
 'topologyVersion': {'processId': ObjectId('6604ee5a50065983486d202e'),
  'counter': 0},
 'maxBsonObjectSize': 16777216,
 'maxMessageSizeBytes': 48000000,
 'maxWriteBatchSize': 100000,
 'localTime': datetime.datetime(2024, 5, 2, 8, 49, 10, 497000),
 'logicalSessionTimeoutMinutes': 30,
 'connectionId': 26,
 'minWireVersion': 0,
 'maxWireVersion': 21,
 'readOnly': False,
 'ok': 1.0}

In [12]:
connection.server_info()

{'version': '7.0.5',
 'gitVersion': '7809d71e84e314b497f282ea8aa06d7ded3eb205',
 'targetMinOS': 'Windows 7/Windows Server 2008 R2',
 'modules': [],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [7, 0, 5, 0],
 'openssl': {'running': 'Windows SChannel'},
 'buildEnvironment': {'distmod': 'windows',
  'distarch': 'x86_64',
  'cc': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.31.31107 for x64',
  'ccflags': '/nologo /WX /FImongo/platform/basic.h /fp:strict /EHsc /W3 /wd4068 /wd4244 /wd4267 /wd4290 /wd4351 /wd4355 /wd4373 /wd4800 /wd4251 /wd4291 /we4013 /we4099 /we4930 /errorReport:none /MD /O2 /Oy- /bigobj /utf-8 /permissive- /Zc:__cplusplus /Zc:sizedDealloc /volatile:iso /diagnostics:caret /std:c++20 /Gw /Gy /Zc:inline',
  'cxx': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.31.31107 for x64',
  'cxxflags': '/TP',
  'linkflags': '/nologo /DEBUG /INCREMENTAL:NO /LARGEADDRESSAWARE /OPT:REF',
  'target_arch': 'x86_64',


### INSERT

In [13]:
blog_ab.insert_one({'emailid':'jhleeroot@gmail.com'})

InsertOneResult(ObjectId('663353c9ebdf4acc8d982032'), acknowledged=True)

### SELECT

In [23]:
blog_ab.find_one( {'emailid':'jhleeroot@gmail.com'} )

{'_id': ObjectId('5f34e53a2171fe53b403d027'), 'emailid': 'jhleeroot@gmail.com'}

### DELETE

In [24]:
blog_ab.delete_one( {'emailid':'jhleeroot@gmail.com'} )

### SELECT *

In [3]:
blog_logs = blog_ab.find()
for log in blog_logs:
    print(log)

{'_id': ObjectId('5f36362bd30e2ec7d748d6e9'), 'session_ip': '127.0.0.1', 'user_email': 'anonymous', 'access_time': '14/08/2020 15:58:51'}
{'_id': ObjectId('5f36362bd30e2ec7d748d6eb'), 'session_ip': '127.0.0.1', 'user_email': 'anonymous', 'access_time': '14/08/2020 15:58:51'}
{'_id': ObjectId('5f363660d30e2ec7d748d6ed'), 'session_ip': '127.0.0.1', 'user_email': 'anonymous', 'access_time': '14/08/2020 15:59:44'}
{'_id': ObjectId('5f36366dd30e2ec7d748d6ef'), 'session_ip': '127.0.0.1', 'user_email': 'jhleeroot@gmail.com', 'access_time': '14/08/2020 15:59:57'}
{'_id': ObjectId('5f36372b043800e068a3edb9'), 'session_ip': '127.0.0.1', 'user_email': 'anonymous', 'page': 'blog_A.html', 'access_time': '14/08/2020 16:03:07'}
{'_id': ObjectId('5f36372f043800e068a3edbb'), 'session_ip': '127.0.0.1', 'user_email': 'jhleeroot@gmail.com', 'page': 'blog_A.html', 'access_time': '14/08/2020 16:03:11'}
{'_id': ObjectId('5f363734043800e068a3edbd'), 'session_ip': '127.0.0.1', 'user_email': 'jhleeroot@gmail.co